In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# Load the data (csv file)
# data = np.loadtxt('__xid-8214579_2.csv', delimiter=',', skiprows=1, dtype=str)
# # Split the data into training and testing sets, 80% split
# train_data = data[:int(len(data)*0.8)]
# test_data = data[int(len(data)*0.8):]

csv_file = '__xid-8214579_2.csv'

# Load the data (csv file) as tf.data
data = tf.data.experimental.make_csv_dataset(
    csv_file,
    batch_size=32,
    label_name='col',
    na_value="?",
    num_epochs=1,
    ignore_errors=True)
    
train_size = int(0.8 * 1000)
val_size = int(0.2 * 1000)
data = data.shuffle(1000)
train_dataset = data.take(train_size)
val_dataset = data.skip(train_size)
val_dataset = val_dataset.take(val_size)

# Check the data shape and type
print(train_dataset.element_spec)

(OrderedDict([('Age', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('Sex', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('Job', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('Housing', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('Saving_accounts', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('Checking_account', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('Credit_amount', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('Duration', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('Purpose', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('Risk', TensorSpec(shape=(None,), dtype=tf.string, name=None))]), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [7]:
# Define the feature columns

feature_columns = []

age = tf.feature_column.numeric_column("Age")
feature_columns.append(age)

sex = tf.feature_column.categorical_column_with_vocabulary_list(
"Sex", ["male", "female"])
sex = tf.feature_column.indicator_column(sex)
feature_columns.append(sex)

job = tf.feature_column.numeric_column("Job")
feature_columns.append(job)

housing = tf.feature_column.categorical_column_with_vocabulary_list(
"Housing", ["own", "rent", "free"])
housing = tf.feature_column.indicator_column(housing)
feature_columns.append(housing)

saving_accounts = tf.feature_column.categorical_column_with_vocabulary_list(
"saving_accounts", ["little", "moderate", "quite rich", "rich", "NA"])
saving_accounts = tf.feature_column.indicator_column(saving_accounts)
feature_columns.append(saving_accounts)

checking_account = tf.feature_column.categorical_column_with_vocabulary_list(
"Checking account", ["little", "moderate", "rich", "NA"])
checking_account = tf.feature_column.indicator_column(checking_account)
feature_columns.append(checking_account)

credit_amount = tf.feature_column.numeric_column("Credit amount")
feature_columns.append(credit_amount)

duration = tf.feature_column.numeric_column("Duration")
feature_columns.append(duration)

purpose = tf.feature_column.categorical_column_with_vocabulary_list(
"Purpose", ["radio/TV", "education", "furniture/equipment", "car", "business", "domestic appliances", "repairs", "vacation/others"])
purpose = tf.feature_column.indicator_column(purpose)
feature_columns.append(purpose)

# Verify the feature columns
print(feature_columns)

[NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), NumericColumn(key='Job', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Housing', vocabulary_list=('own', 'rent', 'free'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='saving_accounts', vocabulary_list=('little', 'moderate', 'quite rich', 'rich', 'NA'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Checking account', vocabulary_list=('little', 'moderate', 'rich', 'NA'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), NumericColumn(key='Credit amount', shape=(1

In [4]:
# Build the model
# Predict the credit risk
# Credit risk is a binary classification problem
# Credit risk is the last column of the data

model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005),
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

In [5]:
filepath = r"C:\Users\karan\Programming\Python\Credit-Risk-Classifier\Final Model\weights-{epoch:02d}.h5"
checkpoint = ModelCheckpoint(filepath, save_weights_only=False, save_freq='epoch', verbose=1)

# Train the model
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=[checkpoint]
)

# Plot the training and validation loss

# Plot the training and validation accuracy

Epoch 1/50


ValueError: in user code:

    File "c:\Users\karan\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\karan\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\karan\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\karan\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\karan\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\karan\anaconda3\envs\tf\lib\contextlib.py", line 135, in __enter__
        return next(self.gen)

    ValueError: E